### Import libraries

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, ReLU, MaxPool2D, Flatten, Activation, GlobalAveragePooling2D, AveragePooling2D, Dropout, Concatenate
from tensorflow.keras.models import Model

### Conv + BN + Relu

In [18]:
def conv_bn_relu(prev_layer , filters , kernel_size , strides =(1,1) , padding = 'same'):
  x = Conv2D(filters=filters, kernel_size = kernel_size, strides=strides , padding=padding)(prev_layer)
  x = BatchNormalization(axis=3)(x)
  x = Activation(activation='relu')(x)
  return x

### Stem block

In [19]:
def StemBlock(prev_layer):
  """
  Operations done before Inception and resuction blocks
  """
  x = conv_bn_relu(prev_layer, filters = 32, kernel_size=(3,3) , strides=(2,2))
  x = conv_bn_relu(x, filters = 32, kernel_size=(3,3))
  x = conv_bn_relu(x, filters = 64, kernel_size=(3,3))
  x = MaxPool2D(pool_size=(3,3) , strides=(2,2)) (x)
  x = conv_bn_relu(x, filters = 80, kernel_size=(1,1))
  x = conv_bn_relu(x, filters = 192, kernel_size=(3,3))
  x = MaxPool2D(pool_size=(3,3) , strides=(2,2)) (x)

  return x

### Inception blocks

In [20]:
def InceptionBlock_A(prev_layer  , filters):

  branch1 = conv_bn_relu(prev_layer, filters = 64, kernel_size = (1,1))
  branch1 = conv_bn_relu(branch1, filters=96, kernel_size=(3,3))
  branch1 = conv_bn_relu(branch1, filters=96, kernel_size=(3,3))

  branch2 = conv_bn_relu(prev_layer, filters=48, kernel_size=(1,1))
  branch2 = conv_bn_relu(branch2, filters=64, kernel_size=(3,3)) # may be 3*3

  branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding='same') (prev_layer)
  branch3 = conv_bn_relu(branch3, filters = filters, kernel_size = (1,1))

  branch4 = conv_bn_relu(prev_layer, filters=64, kernel_size=(1,1))

  output = Concatenate(axis=3)([branch1 , branch2 , branch3 , branch4])

  return output

In [21]:
def InceptionBlock_B(prev_layer , filters):

  branch1 = conv_bn_relu(prev_layer, filters = filters, kernel_size = (1,1))
  branch1 = conv_bn_relu(branch1, filters = filters, kernel_size = (7,1))
  branch1 = conv_bn_relu(branch1, filters = filters, kernel_size = (1,7))
  branch1 = conv_bn_relu(branch1, filters = filters, kernel_size = (7,1))
  branch1 = conv_bn_relu(branch1, filters = 192, kernel_size = (1,7))

  branch2 = conv_bn_relu(prev_layer, filters = filters, kernel_size = (1,1))
  branch2 = conv_bn_relu(branch2, filters = filters, kernel_size = (1,7))
  branch2 = conv_bn_relu(branch2, filters = 192, kernel_size = (7,1))

  branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding ='same') (prev_layer)
  branch3 = conv_bn_relu(branch3, filters = 192, kernel_size = (1,1))

  branch4 = conv_bn_relu(prev_layer, filters = 192, kernel_size = (1,1))

  output = Concatenate(axis=3)([branch1 , branch2 , branch3 , branch4])

  return output

In [22]:
def InceptionBlock_C(prev_layer):

  branch1 = conv_bn_relu(prev_layer, filters = 448, kernel_size = (1,1))
  branch1 = conv_bn_relu(branch1, filters = 384, kernel_size = (3,3))
  branch1_1 = conv_bn_relu(branch1, filters = 384, kernel_size = (1,3))
  branch1_2 =conv_bn_relu(branch1, filters = 384, kernel_size = (3,1))
  branch1 = Concatenate(axis=3)([branch1_1 , branch1_2])

  branch2 = conv_bn_relu(prev_layer, filters = 384, kernel_size = (1,1))
  branch2_1 = conv_bn_relu(branch2, filters = 384, kernel_size = (1,3))
  branch2_2 = conv_bn_relu(branch2, filters = 384, kernel_size = (3,1))
  branch2 = Concatenate(axis = 3)([branch2_1 , branch2_2])

  branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding='same')(prev_layer)
  branch3 = conv_bn_relu(branch3, filters = 192, kernel_size = (1,1))

  branch4 = conv_bn_relu(prev_layer, filters = 320, kernel_size = (1,1))

  output = Concatenate(axis = 3)([branch1 , branch2 , branch3 , branch4])

  return output

### Reduction blocks

```
# This is formatted as code
```



In [23]:
def ReductionBlock_A(prev_layer):

  branch1 = conv_bn_relu(prev_layer, filters = 64, kernel_size = (1,1))
  branch1 = conv_bn_relu(branch1, filters = 96, kernel_size = (3,3))
  branch1 = conv_bn_relu(branch1, filters = 96, kernel_size = (3,3) , strides=(2,2) ) #, padding='valid'

  branch2 = conv_bn_relu(prev_layer, filters = 384, kernel_size=(3,3) , strides=(2,2) )

  branch3 = MaxPool2D(pool_size=(3,3) , strides=(2,2) , padding='same')(prev_layer)

  output = Concatenate(axis = 3)([branch1 , branch2 , branch3])

  return output

In [24]:
def ReductionBlock_B(prev_layer):

  branch1 = conv_bn_relu(prev_layer, filters = 192, kernel_size = (1,1))
  branch1 = conv_bn_relu(branch1, filters = 192, kernel_size = (1,7))
  branch1 = conv_bn_relu(branch1, filters = 192, kernel_size = (7,1))
  branch1 = conv_bn_relu(branch1, filters = 192, kernel_size = (3,3) , strides=(2,2) , padding = 'valid')

  branch2 = conv_bn_relu(prev_layer, filters = 192, kernel_size = (1,1) )
  branch2 = conv_bn_relu(branch2, filters = 320, kernel_size = (3,3) , strides=(2,2) , padding='valid' )

  branch3 = MaxPool2D(pool_size=(3,3) , strides=(2,2) )(prev_layer)

  output = Concatenate(axis = 3)([branch1 , branch2 , branch3])

  return output

In [25]:
def auxiliary_classifier(prev_Layer):
  x = AveragePooling2D(pool_size=(5,5) , strides=(3,3)) (prev_Layer)
  x = conv_bn_relu(x, filters = 128, kernel_size = (1,1))
  x = Flatten()(x)
  x = Dense(units = 768, activation='relu') (x)
  x = Dropout(rate = 0.2) (x)
  x = Dense(units = 1000, activation='softmax') (x)
  return x

### InceptionV3 Model

In [28]:
def InceptionV3():
  """
  Inceptionv3 Model
  - Factorized 7x7 convolutions.
  - BatchNorm in the Auxillary Classifiers.
  - Label Smoothing

  Model input size : (299, 299, 3)
  """

  input_layer = Input(shape=(299 , 299 , 3))

  x = StemBlock(input_layer)

  x = InceptionBlock_A(prev_layer = x ,filters = 32)
  x = InceptionBlock_A(prev_layer = x ,filters = 64)
  x = InceptionBlock_A(prev_layer = x ,filters = 64)

  x = ReductionBlock_A(prev_layer = x )

  x = InceptionBlock_B(prev_layer = x  , filters = 128)
  x = InceptionBlock_B(prev_layer = x , filters = 160)
  x = InceptionBlock_B(prev_layer = x , filters = 160)
  x = InceptionBlock_B(prev_layer = x , filters = 192)

  Aux = auxiliary_classifier(prev_Layer = x)

  x = ReductionBlock_B(prev_layer = x)

  x = InceptionBlock_C(prev_layer = x)
  x = InceptionBlock_C(prev_layer = x)

  x = GlobalAveragePooling2D()(x)
  x = Dense(units=2048, activation='relu') (x)
  x = Dropout(rate = 0.2) (x)
  x = Dense(units=1000, activation='softmax') (x)

  # model = Model(inputs = input_layer , outputs = [x , Aux] , name = 'Inception-V3')
  model = Model(inputs = input_layer , outputs = x , name = 'Inception-V3')


  return model


In [27]:
model = InceptionV3()
model.summary()

Model: "Inception-V3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_81 (Conv2D)          (None, 150, 150, 32)         896       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_81 (Ba  (None, 150, 150, 32)         128       ['conv2d_81[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_81 (Activation)  (None, 150, 150, 32)         0         ['batch_normalizati